# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "glm-4v-9b" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156825 Columns: 14
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: aarch64-apple-darwin20.0.0 (64-bit)
Running under: macOS 15.5

Matrix products: default
BLAS/LAPACK: /Users/zory/miniforge3/envs/py310/lib/libopenblas.0.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] car_3.1-3              carData_3.0-5          emmeans_1.11.0        
[4] marginaleffects_0.25.1 readr_2.1.5            lme4_1.1-35.5         
[7] Matrix_1.6-5          

loaded via a namespace (and not attached):
 [1] bit_4.6.0          jsonlite_2.0.0     compiler_4.3.3     crayon_1.5.3      
 [5] tidyselect_1.2.1   Rcpp_1.0.14        IRdisplay_1.1      parallel_4.3.3    
 [9] splines_4.3.3      boot_1.3-31        uuid_1.2-1         fastmap_1.2.0     
[13] IRkernel_1.3.2     lattice_0.22-6     R6_2.6.1           generics_0.1.4    
[17] Formula_1.2-5      MASS_7.3-60

# Fitting models

In [3]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0355467 (tol = 0.002, component 1)"


In [4]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

boundary (singular) fit: see help('isSingular')



In [5]:
model3 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [6]:
model <- model3

# Interpretation

In [7]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  6195.0   6244.8  -3090.5   6181.0     8993 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.99492 -0.13328 -0.06035  0.15027  3.09692 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 25.23    5.023   
Number of obs: 9000, groups:  Stimulus_ID, 900

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.5260     0.3748   1.404  0.16043    
Angleleft                           -0.3161     0.4879  -0.648  0.51703    
Angleright                          -0.2926     0.4889  -0.598  0.54954    
scale(Proximity, scale = FALSE)     -0.725

In [8]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$`Pr(>|z|)` <- format(res$`Pr(>|z|)`, scientific = FALSE)
res$CI_low <- res$Estimate - 1.96 * res$`Std. Error`
res$CI_high <- res$Estimate + 1.96 * res$`Std. Error`
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),CI_low,CI_high,effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5260132,0.3747527,1.4036275,0.1604298093792,-0.2085021,1.2605285,5.340172,0.09850117
Angleleft,-0.3161462,0.4879364,-0.6479251,0.5170334015169,-1.2725015,0.6402090,5.340172,-0.05920150
Angleright,-0.2925773,0.4888882,-0.5984545,0.5495367089952,-1.2507981,0.6656435,5.340172,-0.05478800
"scale(Proximity, scale = FALSE)",-0.7257426,0.2489979,-2.9146534,0.0035608378311,-1.2137785,-0.2377067,5.340172,-0.13590248
"scale(n_candidates, scale = FALSE)",-1.4594303,0.2936401,-4.9701329,0.0000006690701,-2.0349649,-0.8838957,5.340172,-0.27329276
ActorY,-1.3476203,0.4176765,-3.2264689,0.0012532784417,-2.1662663,-0.5289743,5.340172,-0.25235523


In [9]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.034456034,0.04037645,-0.8533696,0.3934544,1.3457317,-0.11359242,0.04468036
(right) - (front),-0.025989962,0.03993535,-0.6508009,0.5151750,0.9568655,-0.10426181,0.05228189
(right) - (left),0.008466072,0.03126489,0.2707853,0.7865562,0.3463783,-0.05281199,0.06974413


In [10]:
avg_predictions(model, by="Proximity")

Proximity,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.5394280,0.02598580,20.75857,1.025672e-95,315.5466,0.4884968,0.5903593
2,0.4950284,0.01594441,31.04715,1.246547e-211,700.6089,0.4637779,0.5262788
3,0.4471912,0.02225379,20.09506,8.151388e-90,295.9465,0.4035746,0.4908078


In [11]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.7257426,0.2490005,-2.914623,0.003561179,8.133429,-1.213775,-0.2377106,-0.3880239,-0.388169,-0.3880964


In [12]:
avg_predictions(model, by="n_candidates")

n_candidates,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0.5428281,0.02239683,24.23683,9.104532e-130,428.6641,0.4989311,0.5867250
3,0.4613454,0.01620656,28.46659,3.037684e-178,589.7002,0.4295811,0.4931096
4,0.3676835,0.02142979,17.15759,5.515654e-66,216.7837,0.3256819,0.4096851


In [13]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-1.45943,0.2936426,-4.970091,6.692156e-07,20.51103,-2.034959,-0.8839014,-0.3879505,-0.3882424,-0.3880964


In [14]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.1010455,0.03026941,-3.338207,0.0008432096,10.21182,-0.1603725,-0.0417186


The below code is for generating the table in the paper.

In [15]:
stim_id_var_val <- if (!is.null(vc_list$Stimulus_ID) && length(vc_list$Stimulus_ID) > 0) vc_list$Stimulus_ID[[1]] else NA_real_
prompt_id_var_val <- if (!is.null(vc_list$Prompt_ID) && length(vc_list$Prompt_ID) > 0) vc_list$Prompt_ID[[1]] else NA_real_
participant_id_var_val <- if (!is.null(vc_list$Participant_ID) && length(vc_list$Participant_ID) > 0) vc_list$Participant_ID[[1]] else NA_real_

intercept_est <- res[res$term == "(Intercept)", "Estimate"]
intercept_se <- res[res$term == "(Intercept)", "Std. Error"]

angleleft_est <- res[res$term == "Angleleft", "Estimate"]
angleleft_se <- res[res$term == "Angleleft", "Std. Error"]
angleleft_p_val <- res[res$term == "Angleleft", "Pr(>|z|)"]
angleleft_eff_size <- res[res$term == "Angleleft", "effect_size"]

angleright_est <- res[res$term == "Angleright", "Estimate"]
angleright_se <- res[res$term == "Angleright", "Std. Error"]
angleright_p_val <- res[res$term == "Angleright", "Pr(>|z|)"]
angleright_eff_size <- res[res$term == "Angleright", "effect_size"]

proximity_term_name <- "scale(Proximity, scale = FALSE)"
if (!proximity_term_name %in% res$term) {
    proximity_term_name <- grep("Proximity", res$term, value = TRUE)[1]
}
proximity_est <- res[res$term == proximity_term_name, "Estimate"]
proximity_se <- res[res$term == proximity_term_name, "Std. Error"]
proximity_p_val <- res[res$term == proximity_term_name, "Pr(>|z|)"]
proximity_eff_size <- res[res$term == proximity_term_name, "effect_size"]

n_candidates_term_name <- "scale(n_candidates, scale = FALSE)"
if (!n_candidates_term_name %in% res$term) {
    n_candidates_term_name <- grep("n_candidates", res$term, value = TRUE)[1]
}
n_obj_est <- res[res$term == n_candidates_term_name, "Estimate"]
n_obj_se <- res[res$term == n_candidates_term_name, "Std. Error"]
n_obj_p_val <- res[res$term == n_candidates_term_name, "Pr(>|z|)"]
n_obj_eff_size <- res[res$term == n_candidates_term_name, "effect_size"]

actory_term_name <- "ActorY" # Assuming 'X' is the reference level for Actor factor
actory_est <- res[res$term == actory_term_name, "Estimate"]
actory_se <- res[res$term == actory_term_name, "Std. Error"]
actory_p_val <- res[res$term == actory_term_name, "Pr(>|z|)"]
actory_eff_size <- res[res$term == actory_term_name, "effect_size"]

# Prepare data for data frame
table_data <- list(
    c("StimulusID", "Variance", sprintf("%.7f", stim_id_var_val)),
    c("ParticipantID", "Variance", sprintf("%.7f", participant_id_var_val)),
    c("PromptID", "Variance", sprintf("%.7f", prompt_id_var_val)),
    c("Effect Size Denom", "", sprintf("%.7f", effect_size_denom_t)),

    c("Intercept", "b", sprintf("%.7f", intercept_est)),
    c("", "SE", sprintf("%.7f", intercept_se)),

    c("Angleleft", "b", sprintf("%.7f", angleleft_est)),
    c("", "SE", sprintf("%.7f", angleleft_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleleft_p_val))),
    c("", "Effect size", sprintf("%.7f", angleleft_eff_size)),

    c("Angleright", "b", sprintf("%.7f", angleright_est)),
    c("", "SE", sprintf("%.7f", angleright_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleright_p_val))),
    c("", "Effect size", sprintf("%.7f", angleright_eff_size)),

    c("Proximity", "b", sprintf("%.7f", proximity_est)),
    c("", "SE", sprintf("%.7f", proximity_se)),
    c("", "p", sprintf("%.7f", as.numeric(proximity_p_val))),
    c("", "Effect size", sprintf("%.7f", proximity_eff_size)),

    c("#Object", "b", sprintf("%.7f", n_obj_est)),
    c("", "SE", sprintf("%.7f", n_obj_se)),
    c("", "p", sprintf("%.7f", as.numeric(n_obj_p_val))),
    c("", "Effect size", sprintf("%.7f", n_obj_eff_size)),

    c("ActorY", "b", sprintf("%.7f", actory_est)),
    c("", "SE", sprintf("%.7f", actory_se)),
    c("", "p", sprintf("%.7f", as.numeric(actory_p_val))),
    c("", "Effect size", sprintf("%.7f", actory_eff_size))
)

# Convert to data frame
df_output <- do.call(rbind, lapply(table_data, function(x) data.frame(V1=x[1], V2=x[2], V3=x[3], stringsAsFactors=FALSE)))

# Print only the third column for easy copy-pasting to Excel
write.table(df_output[,3, drop=FALSE], file="", sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

25.2275666
NA
NA
5.3401718
0.5260132
0.3747527
-0.3161462
0.4879364
0.5170334
-0.0592015
-0.2925773
0.4888882
0.5495367
-0.0547880
-0.7257426
0.2489979
0.0035608
-0.1359025
-1.4594303
0.2936401
0.0000007
-0.2732928
-1.3476203
0.4176765
0.0012533
-0.2523552
